What kind of analysis can I actually do? 
  * a basic sum of week over week



#### MVP: 
 * a streamlit dashboard and a tableau dashboard
 * filter by activities
 * **total time spent on activites by week**

#### ideas for this data: 
 * convert the map data to elevation data
 * something with the suffer score
   * suffer per second? suffer per minute? 

#### to do
 * convert start_date column to pandas timestamp 

In [4]:
import pandas as pd
#pd.options.display.max_rows = 999

import time
import datetime as dt
import numpy as np
from pathlib import Path
import requests
import json
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

import streamlit as st
import time

import plotly.express as px
import plotly.graph_objs as go
#import matplotlib.pyplot as plt

pd.options.mode.chained_assignment = None

In [5]:
def get_strava_data(csv_file):
    with open('strava_tokens.json') as json_file:
        strava_tokens = json.load(json_file)

    if strava_tokens['expires_at'] < time.time():
        response = requests.post(
                            url = 'https://www.strava.com/oauth/token',
                            data = {
                                    'client_id': 88709,
                                    'client_secret': '54dab45b88619205ee1e47eb94d0f2d84762a4b2',
                                    'grant_type': 'refresh_token',
                                    'refresh_token': strava_tokens['refresh_token']
                                    }
                        ) 
        new_strava_tokens = response.json() 
        with open('strava_tokens.json', 'w') as outfile:
            json.dump(new_strava_tokens, outfile)
        strava_tokens = new_strava_tokens

    page = 1
    url = "https://www.strava.com/api/v3/activities"
    access_token = strava_tokens['access_token']

    activities = pd.read_csv(csv_file)

    while True:
        r = requests.get(url + '?access_token=' + access_token + '&per_page=200' + '&page=' + str(page))
        r = r.json()
        r_df = pd.DataFrame.from_dict(r, orient='columns')     
               
        # if there are no new values (i.e. activities) found in r_df, just break.  otherwise keep going
        if len([v for v in list(r_df['id']) if v not in list(activities['id'])]) == 0:
            activities.drop_duplicates(subset=['id'], inplace=True)
            activities.sort_values(by='id', ascending=False, inplace=True)
            activities.to_csv(csv_file, index=False)
            return activities

        if (not r):   # if no results then exit loop
            break
            
        activities = pd.concat([r_df, activities])
        page += 1
    
    activities.drop_duplicates(subset=['id'], inplace=True)
    activities.sort_values(by='id', ascending=False, inplace=True)
    activities.to_csv(csv_file, index=False)
    return activities

In [6]:
#strava_activities_clean.columns
get_strava_data('strava_activities.csv')

,resource_state,athlete,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,workout_type,...,average_watts,kilojoules,device_watts,Unnamed: 0.1,Unnamed: 0,athlete.id,athlete.resource_state,map.id,map.summary_polyline,map.resource_state
0,2,"{'id': 23871091, 'resource_state': 1}",Morning Run,18377.8,7926,7971,333.0,Run,Run,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,"{'id': 23871091, 'resource_state': 1}",Evening Golf,6545.8,4693,7772,26.0,Golf,Golf,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,"{'id': 23871091, 'resource_state': 1}",Afternoon Ride,6574.6,1079,1117,70.0,Ride,Ride,10.0,...,143.1,154.4,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,"{'id': 23871091, 'resource_state': 1}",Morning Ride,7364.2,1215,1255,37.0,Ride,Ride,10.0,...,84.3,102.4,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2,"{'id': 23871091, 'resource_state': 1}",Night Ride,6409.2,1064,1079,67.0,Ride,Ride,NaN,...,131.2,139.6,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
802,2,"{'id': 23871091, 'resource_state': 1}",Bridge Weaves,15454.2,2733,2922,117.9,Ride,Ride,10.0,...,NaN,NaN,False,195.0,NaN,NaN,NaN,NaN,NaN,NaN
803,2,"{'id': 23871091, 'resource_state': 1}",Westside to Tilikum Crossing Loop,12585.1,2040,2040,91.4,Ride,Ride,10.0,...,NaN,NaN,NaN,196.0,NaN,NaN,NaN,NaN,NaN,NaN
804,2,"{'id': 23871091, 'resource_state': 1}",Maiden Voyage from Forest Park,8532.9,1518,1570,67.7,Ride,Ride,10.0,...,NaN,NaN,False,197.0,NaN,NaN,NaN,NaN,NaN,NaN
805,2,"{'id': 23871091, 'resource_state': 1}",Evening Ride,8836.2,1565,12724,47.3,Ride,Ride,10.0,...,NaN,NaN,False,198.0,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
def prepare_data():
    strava_activities_raw = pd.read_csv('strava_activities.csv')

    cols_to_drop = ['Unnamed: 0.1', 'athlete', 'resource_state', 'start_date_local', 'timezone', 
    'utc_offset', 'trainer', 'commute', 'type', 'manual', 'visibility', 'flagged', 'gear_id', 'start_latlng', 'end_latlng', 'display_hide_heartrate_option',
    'upload_id_str', 'external_id', 'from_accepted_tag', 'total_photo_count', 'athlete.resource_state', 'map.id', 'map.summary_polyline', 'map.resource_state', 
    'average_watts', 'kilojoules', 'device_watts', 'photo_count', 'heartrate_opt_out', 'upload_id', 'athlete.id', 'workout_type',
    'has_heartrate', 'location_city', 'location_state', 'location_country', 'private', 'has_kudoed', 'Unnamed: 0']

    strava_activities_clean = strava_activities_raw.drop(labels=cols_to_drop, axis=1)

    # meters to miles
    strava_activities_clean['distance'] = strava_activities_clean.distance * .000621

    # search results for my pace (pick a pace, any pace) and see my suffering go down over time
    # might be a fun metric to plot over time. perhaps my suffering per minute has gone down as ive gotten fitter
    strava_activities_clean['SPM'] = strava_activities_clean.suffer_score / (strava_activities_clean.moving_time / 60)

    rename = {'start_date': 'start_time'}
    strava_activities_clean.rename(rename, axis=1, inplace=True)

    # convert start_time to pandas Timestamp object. then we can access seconds, minutes, hours, etc. and can add the moving_time
    strava_activities_clean['start_time'] = pd.to_datetime(strava_activities_clean['start_time'])

    # have to manually adjust one activity; the run was on 6/12 not 6/13
    strava_activities_clean.loc[strava_activities_clean.id == 7299973019, 'start_time'] = pd.Timestamp(year=2022, month=6, day=12, hour=2, minute=8, second=36, tz='UTC')

    strava_activities_clean.set_index('start_time', inplace=True)

    return strava_activities_clean

In [8]:
activities = prepare_data()

In [9]:
runs = activities[activities['sport_type'] == 'Run']
runs_2022 = runs[runs.index.year == 2022]

In [10]:
runs_2022.head(3)

,name,distance,moving_time,elapsed_time,total_elevation_gain,sport_type,id,achievement_count,kudos_count,comment_count,...,max_speed,average_cadence,average_temp,average_heartrate,max_heartrate,elev_high,elev_low,pr_count,suffer_score,SPM
start_time,,,,,,,,,,,,,,,,,,,,,
2022-08-07 17:14:37+00:00,Morning Run,11.412614,7926,7971,333.0,Run,7600095117,17,5,0,...,5.378,85.6,27.0,152.9,169.0,319.8,125.6,2,207.0,1.566995
2022-08-03 02:23:45+00:00,Evening Run,3.110341,1374,1374,31.0,Run,7573461254,3,4,1,...,4.463,87.9,29.0,163.0,177.0,83.6,63.2,3,51.0,2.227074
2022-08-01 18:56:46+00:00,Lunch Run,6.258065,3544,3544,74.0,Run,7566246906,0,2,0,...,4.440,83.5,30.0,147.5,159.0,83.8,48.0,0,72.0,1.218962


Miles Ran YTD

In [11]:
miles_run = runs_2022.distance.sum()

Elevation gain YTD

In [12]:
elev_gain = runs_2022.total_elevation_gain.sum()

Miles per day: 

In [13]:
mpd = runs_2022.distance.sum() / runs_2022.index[0].day_of_year

In [14]:
col1, col2, col3 = st.columns(3)
col1.metric("Miles Ran", miles_run, "-$1.25")
col2.metric("Elevation Gain", elev_gain, "0.46%")
col3.metric("Miles Per Day", mpd, "+4.87%")

2022-08-08 20:34:38.567 
  command:

    streamlit run c:\ProgramData\Miniconda3\envs\fitenv\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [17]:
# my 6/12 run is showing up as 6/13

st = strava_activities_clean[strava_activities_clean['sport_type'] == 'Run'].resample('W-Mon', closed='left').distance.sum().tail(12)
#st = strava_activities_clean[strava_activities_clean['sport_type'] == 'Run'].resample('M', closed='left').distance.sum().tail(15)

#st.index = st.index.strftime('%B %Y')
st.index = st.index.strftime('Week of %A, %b %d')

In [27]:
choices = {'W-Mon': '%A, %b %d %Y', 'M': '%B %Y', 'Y': '%Y'}


In [26]:
#list(choices.keys())#['W-Mon']
for choice in choices:
    print(choice, choices[choice])

choices[choice]

W-Mon Week of %A, %b %d %Y
M Month of %B %Y
Y Total for %Y


'Total for %Y'

In [31]:
hunger = ['w', 'm', 'y']

In [33]:
hunger.index('w')

0

In [30]:
choices = {'W-Mon': '%A, %b %d %Y', 'M': '%B %Y', 'Y': '%Y'}

for choice in choices: 
    df_slice = activities[activities['sport_type'] == 'Run'].resample(choice, closed='left').distance.sum().tail(5)
    df_slice.index = df_slice.index.strftime(choices[choice])
    print(df_slice)
    print()

start_time
Monday, Jul 11 2022    41.725922
Monday, Jul 18 2022    16.123271
Monday, Jul 25 2022    17.956401
Monday, Aug 01 2022    15.639016
Monday, Aug 08 2022    20.781020
Name: distance, dtype: float64

start_time
April 2022     132.981437
May 2022       137.867775
June 2022      141.490814
July 2022      107.788212
August 2022     20.781020
Name: distance, dtype: float64

start_time
2018      11.957790
2019      18.698496
2020     470.972734
2021    1491.678825
2022     909.215291
Name: distance, dtype: float64



### Strava Mobile Line Graph
Characteristics: 
* 12 weeks
* Select your activity at the top
* Header: Week of [Month day] - [Month day]
* Subheaders: 
  * Distance
  * Time in xh xxm
  * Elevation

In [61]:
run_activities = strava_activities_clean[strava_activities_clean['sport_type'] == 'Run']
run_by_week = run_activities.resample('W-Mon', closed='left')
run_by_week_agg = run_by_week.agg({'distance': 'sum', 'moving_time': 'sum', 'total_elevation_gain': 'sum'})
run_by_week_agg['total_elevation_gain'] = run_by_week_agg['total_elevation_gain'] * 3.28084
run_by_week_agg.moving_time = run_by_week_agg.moving_time.apply(to_time_readout)
#run_by_week_agg.moving_time = pd.to_datetime(run_by_week_agg.moving_time)


In [56]:
run_by_week_agg.moving_time = run_by_week_agg.moving_time.apply(to_time_readout)

In [52]:
to_time_readout(26363)

'7h 19m'

In [62]:
run_by_week_agg

,distance,moving_time,total_elevation_gain
start_time,,,
2018-08-06 00:00:00+00:00,11.957790,2h 3m,212.598432
2018-08-13 00:00:00+00:00,0.000000,0h 0m,0.000000
2018-08-20 00:00:00+00:00,0.000000,0h 0m,0.000000
2018-08-27 00:00:00+00:00,0.000000,0h 0m,0.000000
2018-09-03 00:00:00+00:00,0.000000,0h 0m,0.000000
...,...,...,...
2022-06-20 00:00:00+00:00,21.896833,3h 18m,1348.425240
2022-06-27 00:00:00+00:00,42.420821,7h 1m,2772.309800
2022-07-04 00:00:00+00:00,26.436405,4h 11m,2742.782240


In [60]:
def to_time_readout(element):
    time = element / 60 / 60
    hrs = int(time)
    mins = (time - hrs) * 60
    return f'{int(round(hrs, 0))}h {int(mins)}m'


In [59]:
int(7.99)

7

In [50]:
(7.32 - round(26363 / 60 / 60, 0)) * 60

19.200000000000017

In [73]:
run_by_week_agg.moving_time = run_by_week_agg.moving_time.apply(lambda x: y := x, f'{int(round(x, 0))}h {int(round((y - round(y)) * 60))}m')

SyntaxError: invalid syntax (2495979164.py, line 1)

In [31]:
run_by_week_agg.moving_time[4].minutes

AttributeError: 'Timedelta' object has no attribute 'minutes'

In [25]:
# week of 7/11 should be 7 hrs 19m in time
#run_by_week_agg.moving_time = dt.timedelta(seconds=run_by_week_agg.moving_time)


In [8]:
f'{int(round(time_wk, 0))}h {int(round((time_wk - round(time_wk)) * 60))}m'

'7h 19m'

In [21]:
print(dt.timedelta(seconds=26363))

7:19:23


In [58]:
# run_by_week_display = 
#run_by_week_agg.distance.round(2)
#run_by_week_agg.total_elevation_gain.round().astype(int)


In [5]:
time_wk = 26363/60/60

In [32]:
run_activities.iloc[[3]]

,name,distance,moving_time,elapsed_time,total_elevation_gain,sport_type,id,achievement_count,kudos_count,comment_count,...,average_speed,max_speed,average_heartrate,max_heartrate,elev_high,elev_low,pr_count,suffer_score,average_cadence,SPM
start_time,,,,,,,,,,,,,,,,,,,,,
2022-07-06 11:26:08+00:00,Lunch Run,4.038177,2164,2172,47.0,Run,7426810587,1,3,0,...,3.005,4.28,140.4,153.0,84.2,62.8,1,23.0,86.0,0.637708


start_time
2018-08-06 00:00:00+00:00     213
2018-08-13 00:00:00+00:00       0
2018-08-20 00:00:00+00:00       0
2018-08-27 00:00:00+00:00       0
2018-09-03 00:00:00+00:00       0
                             ... 
2022-06-20 00:00:00+00:00    1348
2022-06-27 00:00:00+00:00    2772
2022-07-04 00:00:00+00:00    2743
2022-07-11 00:00:00+00:00    2936
2022-07-18 00:00:00+00:00     197
Freq: W-MON, Name: total_elevation_gain, Length: 207, dtype: int32

In [33]:
# 154 ft
run_activities.total_elevation_gain[3] * 3.28084

154.19948

In [37]:
from datetime import timedelta

# create timedelta
sample_timedelta = timedelta(
    days=40,
    seconds=3,
    microseconds=2,
    milliseconds=1200,
    minutes=3,
    hours=23,
    weeks=3
)

In [41]:
sample_timedelta.seconds

82984

In [18]:
#st.index = st.index.strftime('%B %Y')
st

start_time
Week of Monday, May 02    41.244522
Week of Monday, May 09    26.253520
Week of Monday, May 16    20.777232
Week of Monday, May 23    43.479315
Week of Monday, May 30    30.362615
Week of Monday, Jun 06    33.258152
Week of Monday, Jun 13    33.822206
Week of Monday, Jun 20    21.896833
Week of Monday, Jun 27    42.420821
Week of Monday, Jul 04    26.436405
Week of Monday, Jul 11    41.725922
Week of Monday, Jul 18     6.080211
Name: distance, dtype: float64

In [9]:
#strava_activities_clean[strava_activities_clean['sport_type'] == 'Run'].head(30)
strava_activities_clean.columns

Index(['name', 'distance', 'moving_time', 'elapsed_time',
       'total_elevation_gain', 'sport_type', 'id', 'achievement_count',
       'kudos_count', 'comment_count', 'athlete_count', 'map', 'average_speed',
       'max_speed', 'average_heartrate', 'max_heartrate', 'elev_high',
       'elev_low', 'pr_count', 'suffer_score', 'average_cadence', 'SPM'],
      dtype='object')

In [16]:
strava_activities_raw.start_latlng[109]

'[45.52732522599399, -122.60262675583363]'

In [106]:
x = 'start_time'
y = 'average_speed'
df = strava_activities_clean[(strava_activities_clean.sport_type == 'Run') & (strava_activities_clean.distance.between(3.99, 8.5))]

fig = px.scatter(data_frame=df, 
                        y=y,   # choice 
                        x=x,
                        #hover_data=['distance', 'calories'], 
                        #labels=dict(x='Date', time='Time', avg_run_cadence='Average Run Cadence', avg_pace='Average Pace'), 
                        title=str(y + ' x ' + x),
                        #size='distance',
                        #color='distance',
                        template='plotly',
                        height=500, 
                        width=800, 
                        #trendline='ols'
                        )

In [107]:
fig.show()

In [76]:
type(strava_activities_clean['start_time'][2])

str

In [66]:
pd_dt_obj

Timestamp('2022-06-28 19:05:39+0000', tz='UTC')

In [65]:
pd_dt_obj + pd.Timedelta(3600, unit='sec')

Timestamp('2022-06-28 20:05:39+0000', tz='UTC')

In [67]:
regular_dt_obj

numpy.datetime64('2022-06-28T19:05:39.000000000')